In [ ]:
# service account to access namespaces and pods list and details
kubectl create serviceaccount cp-api-explorer

# ro permissions - GET/LIST/WATCH namespaces and pods on cluster level
kubectl create clusterrole ns-reader --verb=get,list,watch --resource=namespaces,pods

# bind role(permissions) to account - CLUSTER level
kubectl create clusterrolebinding ns-reader --clusterrole ns-reader --serviceaccount default:cp-api-explorer


error: failed to create serviceaccount: serviceaccounts "cp-api-explorer" already exists
Error from server (AlreadyExists): clusterroles.rbac.authorization.k8s.io "ns-reader" already exists
error: failed to create clusterrolebinding: clusterrolebindings.rbac.authorization.k8s.io "ns-reader" already exists


[doc](https://nieldw.medium.com/curling-the-kubernetes-api-server-d7675cfc398c)

In [ ]:
# account has secret(s)
kubectl get serviceaccount cp-api-explorer -o json

{
    "apiVersion": "v1",
    "kind": "ServiceAccount",
    "metadata": {
        "creationTimestamp": "2022-03-31T13:04:26Z",
        "name": "cp-api-explorer",
        "namespace": "default",
        "resourceVersion": "501",
        "uid": "95fb1392-b9bd-4f32-ad85-577997e34cf1"
    },
    "secrets": [
        {
            "name": "cp-api-explorer-token-sbfpk"
        }
    ]
}


In [ ]:
# list secrets
kubectl get secrets

NAME                          TYPE                                  DATA   AGE
cp-api-explorer-token-sbfpk   kubernetes.io/service-account-token   3      78m
default-token-8qtt8           kubernetes.io/service-account-token   3      79m


In [ ]:
# we know which from SA, right
$serviceAccount = "cp-api-explorer"
# get right secret
$secretName = kubectl get serviceaccount $serviceAccount -o json | ConvertFrom-Json `
    | % { $_.secrets } `
    | ? { $_.name.startsWith("$($serviceAccount)-token-") } `
    | select -First 1 | % { $_.name }
$secretName
# secret from secretName
$token = kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
# ca.crt
$caCrtFilename = "./k8sca.crt"
kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
gc $caCrtFilename

cp-api-explorer-token-sbfpk
-----BEGIN CERTIFICATE-----
MIIDBjCCAe6gAwIBAgIBATANBgkqhkiG9w0BAQsFADAVMRMwEQYDVQQDEwptaW5p
a3ViZUNBMB4XDTIxMTIxODIxMDIyN1oXDTMxMTIxNzIxMDIyN1owFTETMBEGA1UE
AxMKbWluaWt1YmVDQTCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAMD5
iYNQZEwd7zXtrLY5CV2w2qgyD7xtsnr06c9u0PEYgXEEeQ8iEpi9dNhtOFiahned
mWIikylDD+K0vVba7Hm3U11QPSSAiFwhuTt0Kuhs7dpqKr8dAeOakjNTQLPOFHwW
Qs231Aeme2YKDtpF4w7VbQlOS2iQUvcr8+1JerTVXG4LuphImMUdOGj7ACD+2su3
4J4lRGTlQ9wCiMeAcdya4GEmX3kvMuPNQlZ3J+tO+TqPFHyAoyvNhyy3jqT0wHa7
8VoSQ0lzs/N08qQvdM9eWbmNBKNAEzGdFGoDsQ3h9lbZxfiG2TUTNRQuQbLLbfjZ
mLc5/h7SAEz+vGjSQJcCAwEAAaNhMF8wDgYDVR0PAQH/BAQDAgKkMB0GA1UdJQQW
MBQGCCsGAQUFBwMCBggrBgEFBQcDATAPBgNVHRMBAf8EBTADAQH/MB0GA1UdDgQW
BBSU+f/S+BIFll65N9lYv9ljIo+kGDANBgkqhkiG9w0BAQsFAAOCAQEAHlRd3NHJ
NdslzHem8tnB4mI36pcKemaVWgKh4AoMU9wk+yDfjmVjtUQh5ABrwOaNYL2b/G/W
iiVP0+qrhWHzswu3SqrnC2HW8mM1GgZzO0LI6nspudAfNDFtolEdFx+s3LsmlC+2
KsZVH+cvd+nFDTCyev/05WAp7xQpz8dNlcm/rCM5Pb7ruWTTts964BTeadc/iu2E
qqc/htzwAVYccWc0EKoSoTS5vZzhk2LBWQ

In [ ]:
# k8s API endpoint (if not in minikube, right? - it depends...)
$server = kubectl -n default get endpoints kubernetes --no-headers | awk '{ print $2 }'
$apiServerUrl = "https://$($server)"
$apiServerUrl

https://192.168.64.6:8443


In [ ]:
# with minikube our kube config has server address+port
kubectl config view -o json | jq '.clusters[] | .name'

# kubectl config view -o json | jq '.clusters[] | select(.name == ""minikube"")'

kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'

$apiServerUrl = kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'

"api-crc-testing:6443"
"minikube"
"https://192.168.64.6:8443"


In [ ]:
# run few web servers
1..10 | % { kubectl run "web-$($_)" --image nginx }

In [ ]:
# what does one pod look like?
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq '.items[0]'

{
  "metadata": {
    "name": "middleware-service",
    "namespace": "default",
    "uid": "a501ace3-7857-4047-9bb0-c38058e1848e",
    "resourceVersion": "1286",
    "creationTimestamp": "2022-03-31T13:20:22Z",
    "labels": {
      "app": "nginx",
      "environment": "production"
    },
    "annotations": {
      "kubectl.kubernetes.io/last-applied-configuration": "{\"apiVersion\":\"v1\",\"kind\":\"Pod\",\"metadata\":{\"annotations\":{},\"labels\":{\"app\":\"nginx\",\"environment\":\"production\"},\"name\":\"middleware-service\",\"namespace\":\"default\"},\"spec\":{\"containers\":[{\"image\":\"nginx\",\"name\":\"web\",\"volumeMounts\":[{\"mountPath\":\"/usr/share/nginx/html\",\"name\":\"shared-data\"}]},{\"args\":[\"-c\",\"date | tee /pod-data/index.html; . /usr/local/main/k8s-api-client.sh\"],\"command\":[\"/bin/bash\"],\"image\":\"ubuntu\",\"name\":\"worker\",\"volumeMounts\":[{\"mountPath\":\"/pod-data\",\"name\":\"shared-data\"},{\"mountPath\":\"/usr/local/main\",\"name\":\"main-

In [ ]:
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq '.items[] | .metadata.name'

"middleware-service"
"web-1"
"web-10"
"web-2"
"web-3"
"web-4"
"web-5"
"web-6"
"web-7"
"web-8"
"web-9"


In [ ]:
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq '.items[] | .status.podIPs[0].ip'

"172.17.0.13"
"172.17.0.3"
"172.17.0.12"
"172.17.0.4"
"172.17.0.5"
"172.17.0.6"
"172.17.0.7"
"172.17.0.8"
"172.17.0.9"
"172.17.0.10"
"172.17.0.11"
